In [ ]:
import numpy as np
import pandas as pd
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import os


In [ ]:
print(os.listdir("../input"))

In [ ]:
stop = stopwords.words("english")
for char in [',','.',"'",'"','-', '(',')',':','?','/','>','<',"''", 'br', '\\','...']:
    stop.append(char)

In [ ]:
train = pd.read_csv('../input/labeledTrainData.tsv', delimiter='\t')
test = pd.read_csv('../input/testData.tsv', delimiter='\t')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
len(train)

In [ ]:
len(nltk.word_tokenize(train['review'][0]))

In [ ]:
review_sentiment = []
for id_ in train.index:
    review_sentiment.append([nltk.word_tokenize(train['review'][id_]), train['sentiment'][id_]])

In [ ]:
review_sentiment = review_sentiment[:5000]

In [ ]:
random.shuffle(review_sentiment)

In [ ]:
bag_of_words = []
for (review,sentiment) in review_sentiment:
    for word in review:
        if word not in stop:
            bag_of_words.append(word)

In [ ]:
len(bag_of_words)

In [ ]:
word_FD = nltk.FreqDist(bag_of_words)

In [ ]:
word_FD.most_common(10)

In [ ]:
len(word_FD)

In [ ]:
word_FD_cut = list(word_FD.keys())[:10000]

In [ ]:
featuresets = []
for (review, sentiment) in review_sentiment:
    words = set(word for word in review \
                 if word not in stop)
    features= {}
    for w in word_FD_cut:
        features[w] =  w in words
    featuresets.append([features, sentiment])

In [ ]:
len(featuresets)

In [ ]:
train = featuresets[:4000]
valid = featuresets[4000:]

In [ ]:
clf = nltk.NaiveBayesClassifier.train(train)

In [ ]:
nltk.classify.accuracy(clf, valid)

In [ ]:
clf.show_most_informative_features(10)

In [ ]:
test.head(1)

In [ ]:
test_reviews = []
for id_ in test.index:
    test_reviews.append(nltk.word_tokenize(test['review'][id_]))

In [ ]:
len(test_reviews)

In [ ]:
test_sets = []
for review in test_reviews:
    words = set(word for word in review \
                 if word not in stop)
    features= {}
    for w in word_FD_cut:
        features[w] =  w in words
    test_sets.append(features)

In [ ]:
labels = clf.classify_many(test_sets)

In [ ]:
len(labels)

In [ ]:
submission = pd.DataFrame({"id": test['id'], "sentiment":labels})
submission.to_csv('submission.csv', index=False)